In [4]:
import boto3
from botocore.exceptions import NoCredentialsError, PartialCredentialsError, ClientError
import re
import uuid

class S3Handler:
    def __init__(self, bucketname, region_name='us-east-1'):
        self.bucketname = self.format_bucket_name(bucketname)
        self.s3 = boto3.client('s3', region_name=region_name)
        self.region_name = region_name

    def format_bucket_name(self, bucket_name):
        # Transform to lowercase
        bucket_name = bucket_name.lower()

        # Replace invalid characters
        bucket_name = re.sub(r'[^a-z0-9.-]', '-', bucket_name)

        # Ensure it starts and ends with a letter or number
        bucket_name = re.sub(r'^[^a-z0-9]+', '', bucket_name)
        bucket_name = re.sub(r'[^a-z0-9]+$', '', bucket_name)

        return bucket_name

    def bucket_exists(self):
        try:
            self.s3.head_bucket(Bucket=self.bucketname)
            return True
        except ClientError as e:
            error_code = e.response['Error']['Code']
            if error_code == '404' or error_code == '403':
                return False
            else:
                raise e

    def create_unique_bucket_name(self, base_name):
        while True:
            unique_suffix = uuid.uuid4().hex[:8]
            unique_bucket_name = f"{base_name}-{unique_suffix}"
            if not self.check_bucket_exists(unique_bucket_name):
                return unique_bucket_name

    def check_bucket_exists(self, bucket_name):
        try:
            self.s3.head_bucket(Bucket=bucket_name)
            return True
        except ClientError as e:
            error_code = e.response['Error']['Code']
            if error_code == '404' or error_code == '403':
                return False
            else:
                raise e

    def create_bucket(self):
        try:
            if self.region_name == 'us-east-1':
                self.s3.create_bucket(Bucket=self.bucketname)
            else:
                self.s3.create_bucket(
                    Bucket=self.bucketname,
                    CreateBucketConfiguration={'LocationConstraint': self.region_name}
                )
            return {'status': 'success', 'message': f'Bucket {self.bucketname} created.'}
        except ClientError as e:
            if e.response['Error']['Code'] == 'BucketAlreadyOwnedByYou':
                return {'status': 'success', 'message': f'Bucket {self.bucketname} already exists in your account.'}
            elif e.response['Error']['Code'] == 'BucketAlreadyExists':
                # Bucket exists in someone else's account, create a unique name
                unique_bucket_name = self.create_unique_bucket_name(self.bucketname)
                self.bucketname = unique_bucket_name
                if self.region_name == 'us-east-1':
                    self.s3.create_bucket(Bucket=self.bucketname)
                else:
                    self.s3.create_bucket(
                        Bucket=self.bucketname,
                        CreateBucketConfiguration={'LocationConstraint': self.region_name}
                    )
                return {'status': 'success', 'message': f'Bucket {self.bucketname} created.'}
            else:
                raise e

    def upload_files(self, files):
        responses = []

        if not self.bucket_exists():
            create_response = self.create_bucket()
            if create_response['status'] == 'error':
                return create_response

        for file in files:
            try:
                file_name = file.filename.lower().replace(" ", "-").strip()
                self.s3.upload_fileobj(
                    file,
                    self.bucketname,
                    file_name
                )
                file_url = f"https://{self.bucketname}.s3.amazonaws.com/{file_name}"
                responses.append({
                    'filename': file.filename,
                    'status': 'success',
                    'url': file_url
                })
            except NoCredentialsError:
                responses.append({
                    'filename': file.filename,
                    'status': 'error',
                    'message': 'No AWS credentials found.'
                })
            except PartialCredentialsError:
                responses.append({
                    'filename': file.filename,
                    'status': 'error',
                    'message': 'Incomplete AWS credentials found.'
                })
            except Exception as e:
                responses.append({
                    'filename': file.filename,
                    'status': 'error',
                    'message': str(e)
                })
        return {'status': 'success', 'results': responses}

    # Flask app to use the S3Uploader
    @staticmethod
    def download_file_from_s3(bucket_name, object_key, local_file_path: str):
        """
        This functions downloads the files from s3 bucket.

        :param bucket_name: Name of the S3 bucket
        :param object_key: The key of the file to download
        :param local_file_path: local path of the downloaded. File name with ext is needed.

        """
        try:
            s3 = boto3.client('s3')

            s3.download_file(bucket_name, object_key, local_file_path)
            print(f"File downloaded from S3 bucket {bucket_name} with key {object_key} to {local_file_path}")
        except Exception as e:
            print(f"Error downloading file from S3: {e}")


In [5]:
s3 = S3Handler("myne")
s3.create_bucket()

ClientError: An error occurred (IllegalLocationConstraintException) when calling the CreateBucket operation: The unspecified location constraint is incompatible for the region specific endpoint this request was sent to.